In [1]:
# CART
import numpy as np
import random

In [2]:
class CART(object):
    def __init__(self, x, y, prune=False):
        self.x = x 
        self.y = y 
        # print('build cart...')
        self.branches = self.buildTree(x,y,prune)

    def sign(self,v):
        if v < 0:
            return -1
        else:
            return 1
        
    def buildTree(self,x,y,prune=False):
        branches = []
        if prune:  #只有一层分支
            branch, left, right = self.branchStump(x,y)
            branches.append(branch)
            if sum(left[1]) >= 0:           #表示对left数据集的y值求和
                branches.append([1])
            else:
                branches.append([-1])
            if sum(right[1]) >= 0:
                branches.append([1])
            else:
                branches.append([-1])
            return branches
        else:    #fully grown
            if abs(sum(y)) == len(y):      #二叉决策树终止条件：当前各个分支下包含的所有样本yn都是同类的
                branches.append(y[0])
                return branches 
            else:
                branch, left, right = self.branchStump(x,y)           #二叉决策树前序排列
                branches.append(branch)    
                branches.append(self.buildTree(left[0],left[1]))
                branches.append(self.buildTree(right[0],right[1]))
                return branches 
            
    def branchStump(self, x, y):        #运用desicion stump方法对x和y找到分类出子树的最佳方法！
        # print('13. branch tree once...')
        dimensions = len(x[0])             #这里dimensions为2，相当于特征数
        best_s = True
        best_theta = 0
        best_dim = 0
        best_gini = 100
        for dim in range(dimensions):    
            thetas = np.sort(x[:,dim]) #1/ 每次只选出某一个特征dim用来做decision stump，并且依据该特征对数据集sort。
                                       #2/ 对每个特征，不知道把数据集分到哪一个子分支依据的界限theta是多少，又要每次选出一个theta实验
            ss = [True,False] #3/ 不知道x的该特征是正增益还是负增益。所以s为正遍历一遍，为负遍历一遍
            for i,theta in enumerate(thetas):
                if i > 0:
                    theta = (theta+thetas[i-1])/2 #取每个分割段的中间作为theta
                else:
                    theta = theta/2     #考虑i=0的特殊情况

                for s in ss:
                    gini = self.computeDivideGini(x,y,s,theta,dim) 
                    if gini < best_gini:
                        best_gini = gini 
                        best_s = s
                        best_theta = theta
                        best_dim = dim
        branch = [best_s,best_theta,best_dim]           #结果：得到了最适合的s，theta和dim！！！，最适合的依据是基尼系数最小
        left,right = self.divideData(x,y,best_s,best_theta,best_dim)     #得到了最适合的左右子树
        return branch,left,right 
    
    def hFunc(self, x, s, theta):       #比较x和theta的大小      #s表示方向，可以为true，可以为false
        if s:
            return self.sign(x-theta)            #表示正方向
        else:
            return -self.sign(x-theta)
    
    def divideData(self,x,y,s,theta,dim):     #依据theta这个分割线来具体划分数据集成为两个子数据集
        left_x = []
        left_y = []
        right_x = []
        right_y = []
        for i in range(len(y)):
            if self.hFunc(x[i][dim],s,theta) == -1:              #表示在s方向上x[i][dim]大于theta
                left_x.append(x[i])
                left_y.append(y[i])
            else:
                right_x.append(x[i])
                right_y.append(y[i])
        
        # print('left data:',len(left_y), 'right data:',len(right_y))
        # 注意：下面这种列表list的拼接方法，如a=[np.array(left_x),np.array(left_y)]表示得到了一个长度为2的数组a，
        # a[0]表示left_x，a[1]表示left_y,则a[0][0][0]表示lef_x的第1个元素的第1个特征值
        return [np.array(left_x),np.array(left_y)],[np.array(right_x),np.array(right_y)]

    def computeDivideGini(self,x,y,s,theta,dim):    #依据theta这个分割线得到对应的两个子数据集
        left,right = self.divideData(x,y,s,theta,dim)
        left_gini = self.computeGini(left[0],left[1])
        right_gini = self.computeGini(right[0],right[1])
        return len(left[1])*left_gini+len(right[1])*right_gini

    def computeGini(self,x,y):        #计算基尼指数
        sums = sum(y)             
        lens = len(y)             
        if lens == 0:
            return 0
        pos_num = (sums+lens)/2                 #标签为正的y个数
        neg_num = lens-pos_num                  #标签为负的y个数
        return 1-(pos_num/lens)*(pos_num/lens)-(neg_num/lens)*(neg_num/lens)

    def fit(self,x,branch):             #这里x表示单个数据
        if len(branch) == 3:            #父亲+左子树+右子树的结构
            dim = branch[0][2]          #dim为父亲的最佳dim
            y = self.hFunc(x[dim],branch[0][0],branch[0][1])
            if y == -1:
                return self.fit(x,branch[1])
            else:
                return self.fit(x,branch[2])
        else:   #当最后长度不为3了，表示已经到叶子了，则取出该叶子的值
            return branch[0]    

    def predict(self,x):
        res = []
        for i in range(len(x)):
            res.append(self.fit(x[i],self.branches))
        return np.array(res)

In [3]:
# RandomForest
class RandomForest(object):
    """docstring for RandomForest"""         # T表示有多少棵树
    def __init__(self, x, y, T, prune=False):
         self.x = x
         self.y = y
         self.T = T 
         self.trees = self.buildRF(x,y,T,prune)         #所有的树构成的列表

    def boostrap(self,x,y,N):         #boostrap:对N个数重新排序，创建新数据集
        indexs = [random.randint(0,N) for _ in range(N)]
        return x[indexs], y[indexs]

    def buildRF(self,x,y,T,prune):
        trees = []
        for i in range(T):
            tx, ty = self.boostrap(x,y,len(y)-1)
            trees.append(CART(tx,ty,prune))
        return trees 

    def fit(self,x):         #这里的x是一个数据
        res = []
        for i in range(self.T):
            res.append(self.trees[i].fit(x,self.trees[i].branches))
        if sum(res) >= 0:             #所有数的fit的平均
            return 1
        else:
            return -1 

    def predict(self,x):
        res = []
        for i in range(len(x)):
            res.append(self.fit(x[i]))
        return np.array(res)

In [32]:
if __name__ == '__main__':
    print('load data...')
    
    train_data=np.loadtxt('train_data.txt')
    test_data=np.loadtxt('test_data.txt')
    train_x=train_data[:,0:2]
    train_y=train_data[:,2]
    test_x=test_data[:,0:2]
    test_y=test_data[:,2]    

    cart = CART(train_x,train_y)
    
    train_predict = cart.predict(train_x)
    E_in = 1-sum(train_predict==train_y)/len(train_y)
    print('14. E_in:',E_in)

    predicted = cart.predict(test_x)
    print('15. E_out:',1-sum(predicted==test_y)/len(test_y))
    
    times = 10
    E_in = 0
    E_out = 0
    E_g_in = 0
    T = 300
    for i in range(times):          #随机10次，做10个随机森林，其结果会随着boostrap随机生成的训练集不同而不同
        rf = RandomForest(train_x,train_y,T)

        for tree in rf.trees:
            train_predict = tree.predict(train_x)
            g_in = 1-sum(train_predict==train_y)/len(train_y)
            E_g_in += g_in                       #这是对300棵树*10==3000棵树分别做预测的训练误差的和

        train_predict = rf.predict(train_x)
        e_in = 1-sum(train_predict==train_y)/len(train_y)
        E_in += e_in                #这是随机森林（1个随机森林（300棵树）*10）的训练误差

        predicted = rf.predict(test_x)
        e_out = 1-sum(predicted==test_y)/len(test_y)
        E_out += e_out              #这是随机森林（1个随机森林（300棵树）*10）的验证误差
        print('random forest:',i,'e_in:',e_in,'e_out:',e_out)


    print('16. E_g_in:', E_g_in/T/times)
    print('17. E_in:', E_in/times)
    print('18. E_out:', E_out/times)


    E_in = 0
    E_out = 0
    for i in range(times):
        rf = RandomForest(train_x,train_y,T,True)

        train_predict = rf.predict(train_x)
        e_in = 1-sum(train_predict==train_y)/len(train_y)
        E_in += e_in

        predicted = rf.predict(test_x)
        e_out = 1-sum(predicted==test_y)/len(test_y)
        E_out += e_out
        print('pruned random forest:',i,'e_in:',e_in,'e_out:',e_out)

    print('19. E_in:', E_in/times)
    print('20. E_out:', E_out/times)

load data...
14. E_in: 0.0
15. E_out: 0.126
random forest: 0 e_in: 0.0 e_out: 0.07599999999999996
random forest: 1 e_in: 0.0 e_out: 0.06999999999999995
random forest: 2 e_in: 0.0 e_out: 0.07499999999999996
random forest: 3 e_in: 0.0 e_out: 0.07299999999999995
random forest: 4 e_in: 0.0 e_out: 0.07299999999999995
random forest: 5 e_in: 0.0 e_out: 0.07199999999999995
random forest: 6 e_in: 0.0 e_out: 0.07699999999999996
random forest: 7 e_in: 0.0 e_out: 0.07199999999999995
random forest: 8 e_in: 0.0 e_out: 0.06999999999999995
random forest: 9 e_in: 0.0 e_out: 0.07099999999999995
16. E_g_in: 0.053343333333333874
17. E_in: 0.0
18. E_out: 0.07289999999999995
pruned random forest: 0 e_in: 0.10999999999999999 e_out: 0.15100000000000002
pruned random forest: 1 e_in: 0.08999999999999997 e_out: 0.15100000000000002
pruned random forest: 2 e_in: 0.12 e_out: 0.14800000000000002
pruned random forest: 3 e_in: 0.12 e_out: 0.15900000000000003
pruned random forest: 4 e_in: 0.09999999999999998 e_out: 0.1